In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.layers

/home/dheeraj/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("MNIST_data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
# generating the discriminator fucntion

def discriminator(x_image, reuse=True):
    with tf.variable_scope('discriminator', reuse=tf.AUTO_REUSE):
        
        #defining weights
        
        #Weights for first conv layer
        d_w1 = tf.get_variable('d_w1', [5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))
        #weights for second conv layer
        d_w2 = tf.get_variable('d_w2', [5, 5, 32, 64], initializer=tf.truncated_normal_initializer(stddev=0.02))
        #Weights for fully connected layer
        d_wfc = tf.get_variable('d_wfc', [7*7*64, 5, 1024 ], initializer=tf.truncated_normal_initializer(stddev=0.02))
        #Weights for output layer
        d_wout = tf.get_variable('d_wout', [1024 , 10 ], initializer=tf.truncated_normal_initializer(stddev=0.02))
        
        #defing biases
        
        #bias for first conv layer
        d_b1 = tf.get_variable('d_b1', [32], initializer=tf.constant_initializer(0))
        #bias for second conv layer
        d_b2 = tf.get_variable('d_b2', [64], initializer=tf.constant_initializer(0))
        #bias for fully connected layer
        d_bfc = tf.get_variable('d_bfc', [1024], initializer=tf.constant_initializer(0))
        #bias for output layer
        d_bout = tf.get_variable('d_bout', [10], initializer=tf.constant_initializer(0))
        
        #formation of the neural network layers
        
        #first convolutional layer
        d1 = tf.nn.conv2d(input=x_image, filter=d_w1, strides=[1, 1, 1, 1], padding='SAME')
        d1 = d1 + d_b1
        d1 = tf.nn.relu(d1)
        d1 = tf.nn.avg_pool(d1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        #second convolutonal layer
        d2 = tf.nn.conv2d(input=d1, filter=d_w2, strides=[1, 1, 1, 1], padding='SAME')
        d2 = d2 + d_b2
        d2 = tf.nn.relu(d2)
        d2 = tf.nn.avg_pool(d2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        #fully connected layer
        d_fc = tf.reshape(d2, [-1, 7 * 7 * 64])
        d_fc = tf.matmul(d_fc, d_wfc)
        d_fc = d_wfc + d_bfc
        d_fc = tf.nn.relu(d3)
        
        #output layer
        d_out = tf.matmul(d_wfc, d_wout) + d_bout
        
        return d_out

        
                        

In [12]:
def generator(batch_size, z_dim):
    
    z = tf.truncated_normal([batch_size, z_dim], mean=0, stddev=1, name='z')
    
    #defining weights
    
    #Weights for first deconv layer
    g_w1 = tf.get_variable('g_w1', [z_dim, 3136], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #Weights for second deconv layer
    g_w2 = tf.get_variable('g_w2', [3, 3, 1, z_dim/2], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #Weights for third deconv layer
    g_w3 = tf.get_variable('g_w3', [3, 3, z_dim/2, z_dim/4], dtype=tf.float32, 
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    # Final convolution with one output channel
    g_w4 = tf.get_variable('g_w4', [1, 1, z_dim/4, 1], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
     
    #Defining biases
    
    #bias for first deconv layer
    g_b1 = tf.get_variable('g_b1', [3136], initializer=tf.truncated_normal_initializer(stddev=0.02))
    #bias for second deconv layer
    g_b2 = tf.get_variable('g_b2', [z_dim/2], initializer=tf.truncated_normal_initializer(stddev=0.02))
    #bias for third deconv layer
    g_b3 = tf.get_variable('g_b3', [z_dim/4], initializer=tf.truncated_normal_initializer(stddev=0.02))
    #bias for final deconv layer
    g_b4 = tf.get_variable('g_b4', [1], initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #formation of neural network
    
    g1 = tf.matmul(z, g_w1) + g_b1
    g1 = tf.reshape(g1, [-1, 56, 56, 1])
    g1 = tf.contrib.layers.batch_norm(g1, epsilon=1e-5, scope='bn1')
    g1 = tf.nn.relu(g1)
    
    g2 = tf.nn.conv2d(g1, g_w2, strides=[1, 2, 2, 1], padding='SAME')
    g2 = g2 + g_b2
    g2 = tf.contrib.layers.batch_norm(g2, epsilon=1e-5, scope='bn2')
    g2 = tf.nn.relu(g2)
    g2 = tf.image.resize_images(g2, [56, 56])
    
    g3 = tf.nn.conv2d(g2, g_w3, strides=[1, 2, 2, 1], padding='SAME')
    g3 = g3 + g_b3
    g3 = tf.contrib.layers.batch_norm(g3, epsilon=1e-5, scope='bn3')
    g3 = tf.nn.relu(g3)
    g3 = tf.image.resize_images(g3, [56, 56])
    
    #dimensions of g4 would be 28*28*1
    g4 = tf.nn.conv2d(g3, g_w4, strides=[1, 2, 2, 1], padding='SAME')
    g4 = g4 + g_b4
    g4 = tf.sigmoid(g4)
                    
                    
    return g4        

In [15]:
#setting up the losses and optimisations

sess = tf.Session()

batch_size = 50
z_dimensions = 100

x_placeholder = tf.placeholder("float", shape = [None,28,28,1], name='x_placeholder')

Gz = generator(batch_size, z_dimensions)

Dx = discriminator(x_placeholder)

Dg = discriminator(Gz, reuse=tf.AUTO_REUSE)


ValueError: Variable g_w1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-12-ccb8137112f9>", line 9, in generator
    initializer=tf.truncated_normal_initializer(stddev=0.02))
  File "<ipython-input-13-b1a626665d32>", line 10, in <module>
    Gz = generator(batch_size, z_dimensions)
  File "/home/dheeraj/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
